<a href="https://colab.research.google.com/github/omareldalil4/LSTM_Example/blob/main/LSTM_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class LSTMCell:
    def __init__(self, input_dim, hidden_dim, W_f=None, U_f=None, b_f=None, W_i=None, U_i=None, b_i=None, W_c=None, U_c=None, b_c=None, W_o=None, U_o=None, b_o=None):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.W_f = W_f if W_f is not None else np.random.randn(hidden_dim, input_dim)
        self.U_f = U_f if U_f is not None else np.random.randn(hidden_dim, hidden_dim)
        self.b_f = b_f if b_f is not None else np.zeros((hidden_dim,))
        self.W_i = W_i if W_i is not None else np.random.randn(hidden_dim, input_dim)
        self.U_i = U_i if U_i is not None else np.random.randn(hidden_dim, hidden_dim)
        self.b_i = b_i if b_i is not None else np.zeros((hidden_dim,))
        self.W_c = W_c if W_c is not None else np.random.randn(hidden_dim, input_dim)
        self.U_c = U_c if U_c is not None else np.random.randn(hidden_dim, hidden_dim)
        self.b_c = b_c if b_c is not None else np.zeros((hidden_dim,))
        self.W_o = W_o if W_o is not None else np.random.randn(hidden_dim, input_dim)
        self.U_o = U_o if U_o is not None else np.random.randn(hidden_dim, hidden_dim)
        self.b_o = b_o if b_o is not None else np.zeros((hidden_dim,))

    def forward(self, x_t, h_prev, c_prev):
        f_t = sigmoid(np.dot(self.W_f, x_t) + np.dot(self.U_f, h_prev) + self.b_f)
        i_t = sigmoid(np.dot(self.W_i, x_t) + np.dot(self.U_i, h_prev) + self.b_i)
        c_tilde = tanh(np.dot(self.W_c, x_t) + np.dot(self.U_c, h_prev) + self.b_c)
        c_t = f_t * c_prev + i_t * c_tilde
        o_t = sigmoid(np.dot(self.W_o, x_t) + np.dot(self.U_o, h_prev) + self.b_o)
        h_t = o_t * tanh(c_t)
        return h_t, c_t, f_t, i_t, c_tilde, o_t

class LSTMModel:
    def __init__(self, input_dim, hidden_dim, output_dim, lstm_cell=None, W_y=None, b_y=None):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm_cell = lstm_cell if lstm_cell is not None else LSTMCell(input_dim, hidden_dim)
        self.W_y = W_y if W_y is not None else np.random.randn(output_dim, hidden_dim)
        self.b_y = b_y if b_y is not None else np.zeros((output_dim,))

    def forward(self, inputs):
        h_prev = np.zeros((self.hidden_dim,))
        c_prev = np.zeros((self.hidden_dim,))
        states = []
        for x_t in inputs:
            h_prev, c_prev, f_t, i_t, c_tilde, o_t = self.lstm_cell.forward(x_t, h_prev, c_prev)
            states.append({'h': h_prev, 'c': c_prev, 'f': f_t, 'i': i_t, 'c_tilde': c_tilde, 'o': o_t})
        y_pred = np.dot(self.W_y, h_prev) + self.b_y
        return y_pred, states

input_dim = 1
hidden_dim = 1
output_dim = 1

W_f = np.array([[0.5]])
U_f = np.array([[0.1]])
b_f = np.array([0.0])
W_i = np.array([[0.6]])
U_i = np.array([[0.1]])
b_i = np.array([0.0])
W_c = np.array([[0.7]])
U_c = np.array([[0.3]])
b_c = np.array([0.0])
W_o = np.array([[0.6]])
U_o = np.array([[0.1]])
b_o = np.array([0.2])

custom_cell = LSTMCell(input_dim, hidden_dim, W_f=W_f, U_f=U_f, b_f=b_f,
                       W_i=W_i, U_i=U_i, b_i=b_i,
                       W_c=W_c, U_c=U_c, b_c=b_c,
                       W_o=W_o, U_o=U_o, b_o=b_o)

W_y = np.array([[4.0]])
b_y = np.array([0.0])

model = LSTMModel(input_dim, hidden_dim, output_dim, lstm_cell=custom_cell, W_y=W_y, b_y=b_y)
inputs = [np.array([x]) for x in [1, 2, 3, 4, 5]]
y_pred, states = model.forward(inputs)

print("----- LSTM Model Forward Pass -----")
for t, state in enumerate(states, start=1):
    print(f"Time Step {t}:")
    print(f"  Hidden state (h): {state['h'][0]:.3f}")
    print(f"  Cell state (c):   {state['c'][0]:.3f}")
    print(f"  Forget gate (f):  {state['f'][0]:.3f}")
    print(f"  Input gate (i):   {state['i'][0]:.3f}")
    print(f"  Candidate (c~):   {state['c_tilde'][0]:.3f}")
    print(f"  Output gate (o):  {state['o'][0]:.3f}\n")
print("----- Final Prediction -----")
print(f"y_pred = {y_pred[0]:.3f}")
y_pred_approx = round(y_pred[0])
print(f"y_pred (approx): {y_pred_approx}")


----- LSTM Model Forward Pass -----
Time Step 1:
  Hidden state (h): 0.256
  Cell state (c):   0.390
  Forget gate (f):  0.622
  Input gate (i):   0.646
  Candidate (c~):   0.604
  Output gate (o):  0.690

Time Step 2:
  Hidden state (h): 0.608
  Cell state (c):   0.984
  Forget gate (f):  0.736
  Input gate (i):   0.773
  Candidate (c~):   0.901
  Output gate (o):  0.806

Time Step 3:
  Hidden state (h): 0.825
  Cell state (c):   1.661
  Forget gate (f):  0.826
  Input gate (i):   0.865
  Candidate (c~):   0.979
  Output gate (o):  0.887

Time Step 4:
  Hidden state (h): 0.921
  Cell state (c):   2.395
  Forget gate (f):  0.889
  Input gate (i):   0.923
  Candidate (c~):   0.996
  Output gate (o):  0.936

Time Step 5:
  Hidden state (h): 0.961
  Cell state (c):   3.184
  Forget gate (f):  0.930
  Input gate (i):   0.957
  Candidate (c~):   0.999
  Output gate (o):  0.964

----- Final Prediction -----
y_pred = 3.843
y_pred (approx): 4
